In [1]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance


from   sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from   sklearn.impute import SimpleImputer
from   sklearn.model_selection import train_test_split, GridSearchCV
from   sklearn.metrics import confusion_matrix , classification_report, accuracy_score, roc_auc_score
from   sklearn.linear_model import LogisticRegression
from   xgboost import XGBClassifier
from   lightgbm import LGBMClassifier
from   imblearn.over_sampling import RandomOverSampler
import pickle

# Maths
import math

# Set the options
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [10]:
# Read data in form of a csv file
df1 = pd.read_csv('./artist.csv')
df2 = pd.read_csv('./track.csv')

# First 5 rows of the dataset
df1.head()


,dates,ids,names,monthly_listeners,popularity,followers,genres,first_release,last_release,num_releases,num_tracks,playlists_found,feat_track_ids
0,2024-04-01,01XYiBYaoMJcNhPokrg0l0,stayc,3049521.0,63,1782552,"k-pop, k-pop girl group",2020,2024,14,2,K-Pop ON! (온),0y0uzuB1HxljAY2j0tLETp
1,2024-04-01,099toTcKJoywTosZr2hHjy,langhorne slim,1060777.0,51,163686,"anti-folk, indie folk, modern folk rock, new a...",2008,2023,20,3,Sunrise Run,40rjypnguxfhPy4Bbv5SWc
2,2024-04-01,09HhJFJqYmvCCwQwj47Ci4,hontoni,119199.0,35,60,NaN,2021,2024,14,1,Meditate to the Sounds of Nature,7CIj6Jk5vKNEgxGNRoAWbC
3,2024-04-01,09rAM5GlzBVG3FODKTlrRu,d'verti,46999.0,29,42,NaN,2022,2024,3,1,Meditate to the Sounds of Nature,"6O9l7A57SQ4Z5VwSKyu0UH, 1y5VTWasp7vgqUxjaKUK0B"
4,2024-04-01,0BvkDsjIUla7X0k6CSWh1I,luke bryan,10460890.0,76,8487512,"contemporary country, country, country road, m...",2007,2023,20,6,Sunrise Run,386DfI5sZ0eJi1i6SZVqLw


In [9]:
df2.head()

,ids,names,popularity,markets,artists,release_date,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,musicalkey,musicalmode,time_signature,count,dates,playlists_found
0,7lznzPjPyRFdcYaD9gN7a3,This Is Shangrila,45.0,152.0,3XR64HmFo4OvexUUNW7TP0,1990-01-01,220706.0,0.004280,0.467,0.774,0.001340,0.1580,-10.617,0.0515,109.529,0.324,4.0,0.0,4.0,1.0,2024-04-01,Grunge Forever
1,3e2fDgC93LGc9Lbdvr6I9k,River Of Deceit,52.0,185.0,0T7JQxpy1Li93vLNirbv0Z,1995-03-07,302693.0,0.111000,0.461,0.246,0.000374,0.0963,-13.487,0.0298,88.446,0.178,0.0,1.0,4.0,1.0,2024-04-01,Grunge Forever
2,0donjzZGsCA2PfPUA4EdMN,Shadow of the Season,46.0,185.0,3Ukr3Ufjg8ygRJv7Ww887f,1992,273813.0,0.000046,0.163,0.937,0.002970,0.3720,-5.502,0.0882,184.482,0.413,0.0,1.0,4.0,1.0,2024-04-01,Grunge Forever
3,3bE5slaVEfaDreqARl6k4M,Yellow Ledbetter,69.0,183.0,1w5Kfo2jwwIPruYS2UWh56,1991,303586.0,0.290000,0.504,0.354,0.000005,0.1570,-11.300,0.0339,140.870,0.292,4.0,1.0,4.0,1.0,2024-04-01,Grunge Forever
4,1LvJ1PFobtIm5DwzRrK0jG,Swallow My Pride,43.0,185.0,1XIIxzmo6BNRR4QkImSdsX,1990-09-13,179800.0,0.423000,0.349,0.869,0.000000,0.7600,-11.481,0.0934,127.440,0.193,4.0,1.0,4.0,1.0,2024-04-01,Grunge Forever
